## konlpy로 synop 명사 추출
## TfidfVectorizer, cosine_similarity

문서간의 거리(유사도)  동일 계산임
1. documnet_distance = (tfidf_matrix * tfidf_matrix.T)
2. cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import nltk
from nltk.corpus import stopwords
from konlpy.tag import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
metadata = pd.read_csv('new_all_data_ver3.csv', encoding='cp949')
del metadata['synop']

In [ ]:
metadata.head(1)

## tfidf 문서간 거리(유사도)

In [ ]:
tfidf = TfidfVectorizer(min_df=10)

# tfidf_matrix : 1890x4396 sparse matrix
tfidf_matrix = tfidf.fit_transform(metadata['n_synop'].values.astype('U'))
documnet_distance = (tfidf_matrix * tfidf_matrix.T)
#문서의 거리 ; cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
tfidf

## 코사인 유사도 점수 계산

In [ ]:
# min_df = 10
print('유사도 분석을 위한 ' + str(documnet_distance.get_shape()[0]) + 'x' + str(documnet_distance.get_shape()[1]) + ' matrix.')
print(documnet_distance.toarray())

In [ ]:
# min_df=10
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_sim)
tfidf_matrix.shape

In [ ]:
# 데이터 셋의 id는 1부터, dataframe의 index는 0부터
#index는 dataframe의 idx
indices = pd.Series(metadata.index, index=metadata['idx'])

In [ ]:
def get_recommendations(idx, cosine_sim=cosine_sim):
    #제목과 일치하는 에피소드 인덱스 가지고옴
    idx = indices[idx]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #비슷한 에피소드 상위 10개
    sim_scores = sim_scores[1:11]
    
    # indices
    epi_indices = [i[0] for i in sim_scores]
    
        
    return metadata['epi'].iloc[epi_indices]

In [ ]:
metadata['epi'][1]

In [ ]:
# index - id 불일치
get_recommendations(1)

In [ ]:
print('episode : ' + metadata['epi'][236])
get_recommendations(236, cosine_sim)

## Recommender  

In [ ]:
def recommender(idx):
    print('episode :'+ str(idx) + ' / ' + metadata['epi'][idx-1])
    
    return get_recommendations(idx, cosine_sim)
    

## 전체 데이터 추천 확인

In [ ]:
for i in metadata['idx']:
    print(recommender(i))
    print('')

## ==== 추천 시스템  ====

In [ ]:
def get_recommendations(idx, cosine_sim=cosine_sim):
    #제목과 일치하는 에피소드 인덱스 가지고옴
    idx = indices[idx]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #비슷한 에피소드 상위 10개
    sim_scores = sim_scores[1:11]
    
    # indices
    epi_indices = [i[0] for i in sim_scores]
    print(epi_indices)
    get_result(idx,epi_indices)
    return metadata['epi'].iloc[epi_indices]

In [ ]:
result_data = []
def get_result(idx, epi_indices):
    recom_set = epi_indices.copy()
    recom_set = [ x+1 for x in recom_set ]
    recom_set.insert(0, idx+1)
    print(recom_set)
    result_data.append(recom_set)

In [ ]:
size = metadata.shape[0]
for i in range(1,size+1):
    get_recommendations(i)
    print('\n')

In [ ]:
index_name = ['index']
for i in range(10):
    index_name.append('reco'+str(i+1))
index_name

In [ ]:
df_result = pd.DataFrame(result_data, columns = index_name)
df_result

In [ ]:
df_result.to_csv('recommender_system_web.csv', encoding='utf-8', index=None)